In [1]:
#import utility modules
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as datetime
import random
import matplotlib.pyplot as plt
%matplotlib inline


#preprocessing 
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import cross_val_score


#models
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from theano import tensor as T
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD

import theano
theano.config.floatX = 'float32'

/Users/Charles/anaconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
/Users/Charles/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/Charles/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be r

In [2]:
#these vars are determined from the xgb f score script
df_n_prep = pd.read_csv('train_numeric.csv',usecols=['Id',
'L3_S32_F3850',
'L0_S0_F0',
'L0_S12_F330',
'L1_S24_F1512',
'L3_S38_F3952',
'L3_S33_F3855',
'L0_S0_F2',
'L3_S33_F3859',
'L3_S33_F3857',
'L1_S24_F1514',
'L0_S4_F104',
'L3_S29_F3351',
'L0_S12_F334',
'L1_S24_F1723',
'L1_S24_F1118',
'L0_S1_F28',
'L0_S0_F20',
'L3_S29_F3339',
'L0_S5_F114',
'L3_S30_F3494',
'L1_S25_F1855',
'L3_S33_F3865',
'L3_S36_F3920',
'L0_S0_F4',
'L2_S26_F3069',
'L0_S2_F36',
'L0_S0_F16',
'L3_S29_F3345',
'L3_S30_F3759',
'L1_S24_F1516',
'L2_S27_F3199',
'L3_S29_F3333',
'L2_S26_F3047',
'L0_S13_F356',
'L3_S30_F3754',
'L2_S26_F3106',
'L3_S38_F3960',
'L2_S27_F3129',
'L3_S38_F3956',
'L0_S9_F160',
'L2_S26_F3117',
'L2_S26_F3121',
'L1_S24_F1581',
'L3_S30_F3804',
'L0_S3_F72',
'L3_S30_F3769',
'L0_S3_F80',
'L3_S30_F3749',
'L2_S26_F3036',
'L0_S12_F332',
'L1_S24_F1518',
'L0_S0_F8',
             'Response'], dtype=np.float32 )

num_chunks_test = pd.read_csv("test_numeric.csv", 
                         usecols=['Id',
             'L3_S32_F3850',
'L0_S0_F0',
'L0_S12_F330',
'L1_S24_F1512',
'L3_S38_F3952',
'L3_S33_F3855',
'L0_S0_F2',
'L3_S33_F3859',
'L3_S33_F3857',
'L1_S24_F1514',
'L0_S4_F104',
'L3_S29_F3351',
'L0_S12_F334',
'L1_S24_F1723',
'L1_S24_F1118',
'L0_S1_F28',
'L0_S0_F20',
'L3_S29_F3339',
'L0_S5_F114',
'L3_S30_F3494',
'L1_S25_F1855',
'L3_S33_F3865',
'L3_S36_F3920',
'L0_S0_F4',
'L2_S26_F3069',
'L0_S2_F36',
'L0_S0_F16',
'L3_S29_F3345',
'L3_S30_F3759',
'L1_S24_F1516',
'L2_S27_F3199',
'L3_S29_F3333',
'L2_S26_F3047',
'L0_S13_F356',
'L3_S30_F3754',
'L2_S26_F3106',
'L3_S38_F3960',
'L2_S27_F3129',
'L3_S38_F3956',
'L0_S9_F160',
'L2_S26_F3117',
'L2_S26_F3121',
'L1_S24_F1581',
'L3_S30_F3804',
'L0_S3_F72',
'L3_S30_F3769',
'L0_S3_F80',
'L3_S30_F3749',
'L2_S26_F3036',
'L0_S12_F332',
'L1_S24_F1518',
'L0_S0_F8'], dtype=np.float32)

In [ ]:
#code to create graphs, the L0_S0_F0 is the most predictive var
for feat in df_n_prep.columns[2:]:
    sns.factorplot(data = df_n_prep,x='L0_S0_F0',y=feat ,hue='Response')

In [3]:
#used to grab other important vars from a script on kaggle 
import pandas as pd
import numpy as np
import xgboost as xgb

DATA_DIR = ""

ID_COLUMN = 'Id'
TARGET_COLUMN = 'Response'

SEED = 0
CHUNKSIZE = 50000
NROWS = 250000

TRAIN_NUMERIC = "train_numeric.csv".format(DATA_DIR)
TRAIN_DATE = "train_date.csv".format(DATA_DIR)

TEST_NUMERIC = "test_numeric.csv".format(DATA_DIR)
TEST_DATE = "test_date.csv".format(DATA_DIR)

FILENAME = "etimelhoods"

train = pd.read_csv(TRAIN_NUMERIC, usecols=[ID_COLUMN, TARGET_COLUMN])
test = pd.read_csv(TEST_NUMERIC, usecols=[ID_COLUMN])

train["StartTime"] = -1
test["StartTime"] = -1


nrows = 0
for tr, te in zip(pd.read_csv(TRAIN_DATE, chunksize=CHUNKSIZE), pd.read_csv(TEST_DATE, chunksize=CHUNKSIZE)):
    feats = np.setdiff1d(tr.columns, [ID_COLUMN])

    stime_tr = tr[feats].min(axis=1).values
    stime_te = te[feats].min(axis=1).values

    train.loc[train.Id.isin(tr.Id), 'StartTime'] = stime_tr
    test.loc[test.Id.isin(te.Id), 'StartTime'] = stime_te

#    nrows += CHUNKSIZE
#    if nrows >= NROWS:
#        break

ntrain = train.shape[0]
train_test = pd.concat((train, test)).reset_index(drop=True).reset_index(drop=False)

train_test['z1'] = train_test[ID_COLUMN].diff().fillna(9999999).astype(int)
train_test['z2'] = train_test[ID_COLUMN].iloc[::-1].diff().fillna(9999999).astype(int)

train_test = train_test.sort_values(by=['StartTime', 'Id'], ascending=True)

train_test['z3'] = train_test[ID_COLUMN].diff().fillna(9999999).astype(int)
train_test['z4'] = train_test[ID_COLUMN].iloc[::-1].diff().fillna(9999999).astype(int)

train_test = train_test.sort_values(by=['index']).drop(['index'], axis=1)
train = train_test.iloc[:ntrain, :]

features = np.setdiff1d(list(train.columns), [TARGET_COLUMN, ID_COLUMN])


In [4]:
X = pd.merge(df_n_prep,train_test,how='left',on='Id')
test2 = pd.merge(num_chunks_test,train_test, how='left',on='Id')

X.drop(['Id','Response_x','Response_y'],axis=1,inplace=True)
test2.drop(['Response','Id'],axis=1,inplace=True)
y_hold = pd.read_csv("train_numeric.csv", index_col=0, usecols=[0,969], dtype=np.float32)
y_train = y_hold['Response']



In [ ]:
## beat .37808
#0.37415
imp = Imputer()
scl = StandardScaler()

X = imp.fit_transform(X)
X = scl.fit_transform(X)
y_hold = pd.read_csv("train_numeric.csv", index_col=0, usecols=[0,969], dtype=np.float32)
y_train = y_hold['Response']

boost = xgb.XGBClassifier(max_depth = 8, learning_rate = 0.11,n_estimators=500,
                         min_child_weight=19, silent=1,objective='binary:logistic')

boost.fit(X,y_train)

t2 = imp.fit_transform(test2)
t2 = scl.fit_transform(t2)
print('Fitted')
pred = boost.predict(t2)
Response = pd.Series(pred)
#doing these to get correct dtypes in 
Response2 = pd.Series(np.where(Response==1,1,0))
Id = pd.read_csv("test_numeric.csv", usecols=['Id'])

sub2 = pd.concat([Id,Response2],names=True,axis=1)
sub2.columns = ['Id','Response']
sub2.to_csv('new_tune_money4.csv',index=False)
sum(sub2.Response)

In [ ]:
#best so far

boost = xgb.XGBClassifier(max_depth = 9, learning_rate = 0.099,n_estimators=700,
                         min_child_weight=15)

In [ ]:
boost.1